In [40]:
# Here define data directory and path to metadata csv file
from operator import itemgetter
import numpy as np
tmp=['A','B','C','A','C']
e,indeces=np.unique(tmp, return_index=True)
h=[]
h.append('/ab/test/this')
h.append('/cb/asdf/this')
h.append('/bf/asdd/this')
h.append('/gh/asdf/this')

ind=[0,3]
print ind
print itemgetter(*ind)(h)

j=['a','b','c','d']
j.index('dd')

[0, 3]
('/ab/test/this', '/gh/asdf/this')


ValueError: 'dd' is not in list

In [23]:
import pandas as pd
import numpy as np
import glob, os
from operator import itemgetter

data_path='/nfs/zorba/DOWNLOADED/FCON/FCP/'
# Below will capture all the structural and functional dicom folders (some are duplicates)
#T1_glob_data = '*/*/A*/*_V*/MPRAGE*/co*nii.gz' # Grabs all 
#T1_glob_data = '*/*/A*/*_V{2,A,_ALG}/MPRAGE*/co*nii.gz' # Grabs all baseline
T1_glob_data = '*/scans/anat_mprage_anonymized/NIfTI/scan*.nii' # Grabs all baseline, later can add the repeat session
func_glob_data = '*/scans/func_rest/NIfTI/*.nii' # Come back and incorporate
os.chdir(data_path)
T1_scans=sorted(glob.glob(T1_glob_data)) # Focus on T1 scans for now
func_scans=sorted(glob.glob(func_glob_data))
print "length of structural scans is: " + str(len(T1_scans))
print "length of functional scans is: " + str(len(func_scans))
# Now get only unique ids for structural scans
T1_sub_ids=[]
for s in T1_scans:
    T1_sub_ids.append(s.split('/')[0])
    
# Now get only unique ids for functional lores resting scans
func_sub_ids=[]
for s in func_scans:
    func_sub_ids.append(s.split('/')[0])

metadata_file = '/nfs/zorba/Dropbox/DOWNLOADED/FCON/guest_7_11_2017_19_55_52.csv'
df = pd.read_csv(metadata_file)
#df['Anoymized ID'] = df['Anoymized ID'].apply(lambda x: unicode(x))
#id_list = df['Anonymized ID'].values.tolist()
df = df.set_index('MR ID')
id_list = df.index.tolist()
subject_dict = {}
count=0

for i in range(len(T1_sub_ids)):
    item=T1_sub_ids[i] # Item is the subject ID "i.e. A00053278"
    T1_path=T1_scans[i]
    #print (item)
    #print file_path
    if item in id_list:
        # Here go ahead and grab the subject's lores REST func_path too
        try:
            func_index = func_sub_ids.index(item)
            func_path  = func_scans[func_index]
        except: 
            func_path = []
        #print id_list.index(item)
        count += 1
        df_index =id_list.index(item)
        age = df.iloc[df_index]['Age']
        sex = df.iloc[df_index]['M/F']
        fs  = df.iloc[df_index]['FieldStrength']
        
        # make sure age and sex are defined
        if sex=="U":
            pass
        else:
            subject_dict[item] = {'age': "", "sex": "", "T1_path":"", "func_path":"", "FS":""}
            subject_dict[item]['age'] = age
            subject_dict[item]['sex'] = sex
            subject_dict[item]['FS'] = fs
            subject_dict[item]['T1_path'] = T1_path
            subject_dict[item]['func_path'] = func_path
       
print (count, len(T1_sub_ids))
print len(subject_dict)

for key, value in subject_dict.items():
    print key  
    print value["T1_path"]
    print value["func_path"]
    print value["age"]
    print value["sex"]
    print value["FS"]
        

length of structural scans is: 869
length of functional scans is: 841
(869, 869)
762
NewYork_sub59796
NewYork_sub59796/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
NewYork_sub59796/scans/func_rest/NIfTI/scan_rest.nii
34.0
M
3T
NewYork_sub02503
NewYork_sub02503/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
NewYork_sub02503/scans/func_rest/NIfTI/scan_rest.nii
9.0
M
3T
Leiden_sub38454
Leiden_sub38454/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Leiden_sub38454/scans/func_rest/NIfTI/scan_rest.nii
21.0
M
3T
Cleveland_sub22935
Cleveland_sub22935/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Cleveland_sub22935/scans/func_rest/NIfTI/scan_rest.nii
39.0
F
3T
Milwaukee_sub16666
Milwaukee_sub16666/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Milwaukee_sub16666/scans/func_rest/NIfTI/scan_rest.nii
47.0
F
3T
Cambridge_sub08723
Cambridge_sub08723/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Cambridge_su

In [65]:
import pyxnat
import os
delete_previous_scans = 1
insert_scans = 1

# Set project name
project_name="FCON_TEST"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# Create project object
project=xnat.select.project(project_name)

# First check if it already exists
if not project.exists():
    project.insert()
    
for key, value in subject_dict.items()[0:5]:
    print key
    print value["T1_path"]
    
    # Insert subject if not already there
    print ("Now insert subject with key: " + key)
    subject = project.subject(key)
    if not subject.exists():
        subject.insert()
    
    # Since this is baseline just do MR1 for now
    session_label = "%s_%s_MR1" % (project_name,key)
    print session_label
    
    experiment=subject.experiment(session_label) #.create(experiments='xnat:mrSessionData')
   
    # Create experiment object, insert if it doesn't already exst
    if not experiment.exists():
        experiment.create() # By default it will make an xnat:mrSessionData experiment which is what we need
    
    # Now insert subject-level attributes (i.e. SEX) and experiment-level attributes (i.e. AGE)
    user_age = int(value['age'])
    if value['sex'] == 'M':
        user_sex = 'Male'
    else:
        user_sex = 'Female'
    
#     sex = 'Male'
#     weight = str(100)
#     height = str(100)
    
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender', user_sex)
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age', str(user_age))
    #subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/height', height)
    
    #model ='SIEMENS MAGNETOM TrioTim syngo MR B17'
    #site ='Hammersmith Hospital'
    fs = value['FS']
    
    #experiment.attrs.set('xnat:imageSessionData/scanner',model)
    experiment.attrs.set('xnat:imageSessionData/fieldStrength',fs)

    # Below for inserting scans
    if insert_scans==1:
        scan_files=[]
        scan_files.append(value['T1_path'])
        if value['func_path'] != []:
            scan_files.append(value['func_path'])
    
        # Delete old scans if desired
        if delete_previous_scans==1:
            allScans=experiment.scans()
            for scan in allScans:
                print "deleting scans"
                scan.delete()

        for ii in range(len(scan_files)):
            if ii==0:
                scan_type='T1'
                scan_name='T1_01'
                slice_size='8 1200'
            elif ii==1:
                scan_type='func_rest'
                scan_name='func_rest_02'
                slice_size='3 267'

            img_reorient='anonymized_reorient.nii.gz'
            # rm img_reorient
            cmd_rm='rm anonymized_reorient.nii.gz temp.nii.gz'
            os.system(cmd_rm)
            #cmd_reorient='fslreorient2std ' + value['T1_path'] + ' ' + img_reorient
            cmd_rename='cp ' + scan_files[ii] + ' temp.nii && gzip temp.nii && mv temp.nii.gz ' + img_reorient # Don't reorient since already including dcm2nii orientation
            os.system(cmd_rename)
            
            sform = subprocess.check_output(['fslorient', '-getsform',img_reorient]).split(' ')
            sform[3] = '90'; sform[7] = '-125'; sform[11] = '-67.75'
            new_sform=' '.join(sform).rstrip()
            cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_reorient + ' && fslorient -setqform ' + new_sform + ' ' + img_reorient
            print cmd_reorient
            os.system(cmd_reorient)
            print subprocess.check_output(['fslorient', '-getsform',img_reorient])

            # Now insert the reoriented image
            # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
            experiment.scan(scan_name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
            img_reorient,
            content=scan_type,
            overwrite=True,
            type=scan_type,  # May need to change this to SPGR, MPRAGE, FLAIR etc?
            format='NIFTI')

            # Create ORIGINAL and SNAPSHOT to view the images in the web browser
            img_png='tmp.png'
            img_gif='tmp.gif'
            img_thumb='tmp_thumb.gif'
            cmd_create_slices='slicer ' + img_reorient + ' -S ' + slice_size + ' ' + img_png
            cmd_convert_gif='convert ' + img_png + ' ' + img_gif
            cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

            os.system(cmd_create_slices)
            os.system(cmd_convert_gif)
            os.system(cmd_convert_thumb)

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
            img_gif,
            content='ORIGINAL',
            overwrite=True,
            format='GIF')

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
            img_thumb,
            content='THUMBNAIL',
            overwrite=True,
            format='GIF')
                                                    

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')

#db.select.project('IXI_TEST').create()
print "ok"
    

NewYork_sub59796
NewYork_sub59796/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Now insert subject with key: NewYork_sub59796
FCON_TEST_NewYork_sub59796_MR1
deleting scans
deleting scans
fslorient -setsform -1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient.nii.gz && fslorient -setqform -1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient.nii.gz
-1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 

fslorient -setsform -1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient.nii.gz && fslorient -setqform -1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient.nii.gz
-1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 

NewYork_sub02503
NewYork_sub02503/scans/anat_mprage_anonymized/NIfTI/scan_mprage_anonymized.nii
Now insert subject with key: NewYork_sub02503
FCON_TEST_NewYork_sub02503_MR1
deleting scans
deleting scans
fslorient -setsform -1 0 0 90 0 1 0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient.nii.gz && fslorient -setqform -1 0 0 90 0 1 0 -1

In [63]:
import subprocess
os.chdir('/nfs/zorba/DOWNLOADED/')
img_name = 'anonymized_reorient2'
sform = subprocess.check_output(['fslorient', '-getsform',img_name]).split(' ')
print "old sform is: " + ' '.join(sform)

sform[3] = '90'
sform[7] = '-125'
sform[11] = '-67.75'

new_sform=' '.join(sform).rstrip()
print "new sform is: " + new_sform
cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_name + ' && fslorient -setqform ' + new_sform + ' ' + img_name
#cmd_reorient = 'fslorient -setsform ' + new_sform + ' ' + img_name
print cmd_reorient
os.system(cmd_reorient)
print subprocess.check_output(['fslorient', '-getsform',img_name])


old sform is: -1 -0 -0 -0 -0 1 -0 -0 0 0 1.2 -0 0 0 0 1 

new sform is: -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1
fslorient -setsform -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient2 && fslorient -setqform -1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 anonymized_reorient2
-1 -0 -0 90 -0 1 -0 -125 0 0 1.2 -67.75 0 0 0 1 

